# Summarizing
このレッスンでは、記事を要約する処理を学習します

## import

In [12]:
import openai
import json

## APIkey設定

In [13]:
json_file = open("./apikey.json", 'r')
api_key = json.load(json_file)
openai.api_key = api_key["apikey"]

## 実行関数

In [14]:
# プロンプトを受け取り、そのプロンプトの完了を返却型する関数
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

以下は要約したい文章です

In [15]:
prod_review = """
娘の誕生日にこのパンダのぬいぐるみをプレゼントしました。
彼女はこのぬいぐるみが大好きで、どこにでも持っていきます。
柔らかくて、とてもかわいいです。
でも、値段の割にはちょっと小さいかな。
同じ値段でもっと大きいのが他にありそうな気がします。
予定より1日早く届いたので、渡す前に確認することができました。
"""

### 要約する

プロンプトを書きます

In [16]:
prompt = f"""
あなたのタスクは、eコマースサイトからの製品レビューの短い要約を生成することです。

以下のレビューを、三重のバックティックで区切り、最大30語で要約してください。

レビュー: ```{prod_review}```
"""
response = get_completion(prompt)
print(response)

要約: 娘が大好きな柔らかくてかわいいパンダのぬいぐるみ。値段に比べて少し小さいが、早く届いた。


レビューの内容を部署ごとに返却したい場合は、どこに注目して要約するかをプロンプトに記載してあげる。
出荷部門にフィードバックをする場合は、商品の発送と配送に重点を置く

In [25]:
prompt = f"""
あなたの仕事は、eコマースサイトの商品レビューの短い要約を作成することです。

レビューの要約を、50文字以内で作成しなさい。
作成する要約には製品の配送に関することだけを書きなさい。

レビュー: ```{prod_review}```
"""
response = get_completion(prompt)
print(response)

配送は予定より1日早く届き、プレゼントの準備ができました。


価格とぬいぐるみの大きさに焦点を当てた場合のプロンプト↓↓

In [29]:
prompt = f"""
あなたの仕事は、eコマースサイトの商品レビューの短い要約を作成することです。

レビューの要約を、50文字以内で作成しなさい。
作成する要約には製品の価格と大きさに関することだけを書きなさい。

レビュー: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

価格は高めだけど、柔らかくてかわいいパンダのぬいぐるみ。
ただ、値段に比べて少し小さいかもしれない。



### 抽出する

In [32]:
prompt = f"""
あなたの仕事は、eコマースサイトの商品レビューから関連情報を抽出し、配送部門にフィードバックすることです。

以下のレビューから、三重引用符で区切られた、配送に関連する情報を抽出しなさい。
30字以内とする。

レビュー: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

配送に関連する情報: 
「予定より1日早く届いた」


### 複数のレビューをまとめる

In [42]:
# 電動歯ブラシ
review_2 = f"""
外観は白一色で清潔感は有る。
高級感は感じられないが、音波式歯ブラシの性能が重要です。
お尻部分にゴムが付いていて縦置きでも滑らない。
横置きでも転がらないように小さな突起が付いています。
充電は付属のUSB専用コードを本体お尻部分ゴムを外し、分かりにくいが中央部丸いゴムを外すと差込口が有ります。
充電中は本体が赤く光り、完了すると(緑色になると取説には記載されているが点きませんでした。)消えます。
音波振動は６つのバージョンが有り、お好みで使用できます。
日本語の説明書も付属していました。耐久性については検証待ちです。
"""

# ドライヤー
review_3 = f"""
ドライヤーが壊れて選ぶのが面倒だったので娘の家のと同じ物を購入しましたが、これからの季節には風量は申し分ないので問題無いですが真冬には熱量が少し不足かなと思います。
前はパナソニックnano eで1300w物だったので100Wでこれだけ熱量が変わるのかな？
と思いましたが髪には良さそうです、サラサラになります。
"""

# イヤホン
review_4 = f"""
私が初めて買ったワイヤレスイヤホンです。
人気があったのと、SONYのワイヤレスイヤホンが気になっていたのでこれに決めました。
長文になります。

まずケース。
前のモデルであるWF-1000XM3から充電回数が1回減った代わりにかなり小型化されています。
充電回数が減ったものの、本体のバッテリー持続時間が延びているのでケースを含めた使用時間は変わりません。
私は使っていませんがワイヤレス充電に対応しているのもいいですね。
あとは本体をケースに収納する際、マグネット方式でスムーズに収納できるのもいい。
ケースに吸い込まれるような感覚はクセになります。
ケースが自立するようになったのもWF-1000XM3から変わったポイントです。

続いて本体。
WF-1000XM3と同じく高級感のあるデザインですね。
マットになっています。
金色の部分(マイク)が特徴的です。
大きさは購入当時はなんとも思いませんでしたが、大きく感じる方もいるかもしれません。
それでもWF-1000XM3よりは小さくなっています。

音質はシンプルにとても良いです。
詳しくないので上手く表現できませんが、バランスがよくて聞きやすい音だと思います。
音場も広い。個人的にはとても気に入っています。

この製品の目玉、ノイズキャンセリング。
後から違う製品を買ってよさがわかりました。
まずエアコンの風の音などは完全といっていいほど消えます。
ノイズキャンセリングは人の声などの高音は軽減しにくい特性があるのですが、
そこで効果を発揮するのが固いスポンジのような特殊なイヤーピース、「ノイズアイソレーションイヤーピース」。とても遮音性がよく、特に高音域で効果を発揮します。代わりに圧迫感のある装着感で、合わない人もいると思います。
冷えると固くなるなど、本当に特殊です。

外音取り込みはとても自然です。
人の声などは若干聞き取りにくいかもしれませんが、イヤホンをしていてもしていないような解放感が得られます。
取り込む量をアプリで20段階で調節できます。私は常時最大にしています。

専用スマートフォンアプリで使用できるイコライザーで低音やボーカルを上げるなど、音をカスタマイズできます。音楽再生アプリにあるイコライザーよりキレイに調整できます。
アプリからDSEE Extremeを有効にするとMP3などの圧縮音源をハイレゾ相当にアップスケーリングできます。細かい音がキレイになります。
いずれも利用するとバッテリー持続時間が短くなります。

私は現時点では対応機器をもっていないのですが、LDACコーデックに対応しているので対応機器をお持ちであればワイヤレスでハイレゾ音源の再生ができます。
対応コーデックはSBC、AAC、LDACです。

バッテリーは本体はノイズキャンセリングONで8時間、OFFで12時間です。
長いですね。
ですが、これは「最大で」なので実際はもっと短くなることがほとんどです。
各種機能を有効にするとバッテリー消費が多くなりますし、通話時は消費が激しいです。
ケースで本体を2回満充電できます。
外出先でバッテリーに困ったことはありません。

通話にも対応しています。
騒がしい環境でも自分の声だけを相手に届けることに長けています。

アプリが使用者の動きを検知してあらかじめ設定した外音コントロール(ノイズキャンセリング・外音取り込み・オフ)に切り替えてくれるアダプティブサウンドコントロールがあります。
使用者の動きは、「止まる」「歩く」「走る」「(電車やバスなどに)乗る」の4つに分類されます。
他にも地図から地点を登録して切り替えることもできるようです。
この機能の使い方の例として、いつも使っている駅を外音取り込みで登録することで、電車に乗っている時、降りる駅が近づくと自動で外音取り込みに切り替わって乗り過ごしを防ぐという使い方があります。
私は自動で切り替わることを好まないのでこの機能は使っていません。
ノイズキャンセリングで周りの音が聞こえない状態でも、自分が話し始めると自動で音楽の停止と外音取り込みへの切り替えを行ってくれ、人と会話ができるスピーク・トゥ・チャット。
便利な機能だと思いますが、感度を3段階のうち最大にしても反応しないことがあり、大事なときに話せないので使っていません。声質にもよるのでしょうか。けっこう大きな声で話しているんですけどね。
わりと最近のアップデートで追加された機能、マルチポイント。
2台の機器と同時接続でき、パソコンで作業していてもスマホの着信があるとそちらに切り替わって通話ができる、なんて使い方ができるようです。
2台の機器のうち、片方は専用アプリが入っている機器である必要があります。
Googleアシスタント、Amazon Alexa、Siriにも対応しています。
スマホを一切さわらずに使えるので使いこなせればかなり便利だと思います。
とてもいい製品ですがこの製品の最大の難点、不具合。
けっこう有名な話ですが、左右のうち片方のバッテリーが極端に減る不具合が多いようです。
それだけではなく友人は異音の初期不良で返品していますし、私もその他症状で修理に出しています。
この製品を購入するときは不具合がないことを祈って購入するしかないです。
発売から2年近く経ってのレビューでした。もうそろそろ発表されそうな次のモデルに期待しています。
"""

reviews = [review_2, review_3, review_4]


In [43]:
for i in range(len(reviews)):
    prompt = f"""
    あなたのタスクは、eコマースサイトからの製品レビューの短い要約を生成することです。
    以下のレビューを、三重のバックティックで区切り、最大30文字で要約してください。

    レビュー: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

0 要約: 外観は清潔感があり、充電はUSBで簡単。音波振動は６つのバージョンがあり、日本語説明書も付属。耐久性は未検証。 

1 要約: ドライヤーが壊れたため同じものを購入。風量は問題ないが真冬には熱量が不足。髪はサラサラに。 

2 SONYのワイヤレスイヤホンは、小型化されたケースと高級感のあるデザインが特徴。音質はバランスがよく、ノイズキャンセリングはエアコンの音も消えるほど。バッテリー持続時間も長く、外音取り込みも自然。ただし、左右のバッテリー不具合があるため注意が必要。 

